In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

from scipy.signal import find_peaks

In [ ]:
file_path = r'D:\MiniSOM clustering\adatok\filtered_and_cropped_data_max_and_min_value_filtered_v3.csv'

df = pd.read_csv(file_path)

In [ ]:
# === PEAK FINDER SETTINGS ===
PEAK_HEIGHT = None        # e.g., 0.5 or None
PEAK_PROMINENCE = 0.6     # how much peak stands out from surroundings
PEAK_DISTANCE = 3         # min frames between peaks
PEAK_WIDTH = None         # min width of peaks in frames
PEAK_THRESHOLD = None     # vertical diff with neighbors
# ===========================

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
sns.set(style="white")
sns.set(style="ticks")

# Get all parent0/parent1 combinations
conditions = df[['itga_krt', 'iso_hypo']].drop_duplicates().reset_index(drop=True)

# Collect average peaks per exp_name for each condition
avg_peaks_per_exp = []

# --- 4 subplots with average_dmap-based hue and Spectral colormap ---

fig, axes = plt.subplots(2, 2, figsize=(16, 12), sharex=True, sharey=True)
axes = axes.flatten()

for idx, row in conditions.iterrows():
    p0, p1 = row['itga_krt'], row['iso_hypo']
    subset = df.query("itga_krt == @p0 and iso_hypo == @p1").copy()
    ax = axes[idx]
    peak_counts = []

    # Convert frame to time_minute if you want time on x-axis
    subset["time_minute"] = subset["time"] / 60

    # Set up color normalization and palette for average_dmap
    norm = plt.Normalize(0, 200)
    palette = sns.color_palette("Spectral", as_cmap=True)
    cmap = plt.cm.Spectral

    # Plot all cell traces colored by average_dmap
    sns.lineplot(
        x="time_minute",
        y="gpr_dF/F0",
        units="unique_cell_id",
        estimator=None,
        hue="average_dmap",
        data=subset,
        palette=palette,
        lw=0.5,
        ax=ax,
        legend=False
    )

    # Find and plot peaks for each cell
    for cell_id, group in subset.groupby('unique_cell_id'):
        group = group.sort_values('frame')
        y = group['gpr_dF/F0'].values
        x = group["time_minute"].values
        avg_dmap = group['average_dmap'].iloc[0]
        color = cmap(norm(avg_dmap))

        peaks, _ = find_peaks(
            y,
            height=PEAK_HEIGHT,
            prominence=PEAK_PROMINENCE,
            distance=PEAK_DISTANCE,
            width=PEAK_WIDTH,
            threshold=PEAK_THRESHOLD
        )

        peak_counts.append({
            'unique_cell_id': cell_id,
            'exp_name': group['exp_name'].iloc[0],
            'peak_count': len(peaks)
        })

        # Overlay peaks as black dots
        ax.plot(x[peaks], y[peaks], 'o', markersize=4, color=color, alpha=0.65, markeredgecolor='black', markeredgewidth=0.15)

    ax.set_title(f"{p0} - {p1}", fontsize=18)
    ax.set_xlabel("Time (min)" if "time_minute" in subset.columns else "Frame")
    ax.set_ylabel("∆F/Fo")
    sns.despine(ax=ax)
    

    ax.set_xlim(0, 15)
    ax.set_ylim(-1.1, 10.5)
    if idx in [2, 3]:
        ax.set_xlabel("Time (min)")
        ax.set_xticks([5, 15])
        ax.set_xticklabels(['5', '15'])
    else:
        ax.set_xticks([5, 15])
        ax.set_xticklabels([])

    # Summary stats
    peak_df = pd.DataFrame(peak_counts)
    avg_per_exp_name = peak_df.groupby('exp_name')['peak_count'].mean().reset_index()
    avg_per_exp_name['condition'] = f"{p0} - {p1}"
    avg_peaks_per_exp.append(avg_per_exp_name)

    overall_avg = avg_per_exp_name['peak_count'].mean()
    print(f"\nOverall average number of peaks for {p0} - {p1}: {overall_avg:.2f}")
    print("Average number of peaks per exp_name:")
    display(avg_per_exp_name)

    

    # # Optional: add colorbar to the first subplot only
    # if idx == 0:
    #     sm = plt.cm.ScalarMappable(cmap="Spectral", norm=norm)
    #     sm.set_array([])
    #     cbar = fig.colorbar(sm, ax=ax, orientation='vertical')
    #     cbar.set_label("Distance from wound (µm)", fontsize=12)

plt.tight_layout()
plt.savefig(r'D:\Peak finder\plots\peak_finder_lineplots_per_condition.pdf', bbox_inches='tight')
plt.show()


# --- Barplot of average number of peaks per exp_name for each condition ---

# Combine all into one DataFrame
avg_peaks_df = pd.concat(avg_peaks_per_exp, ignore_index=True)

avg_peaks_df.to_csv(r'D:\Peak finder\data\avg_peaks_per_exp.csv', index=False)

plt.figure(figsize=(5, 6))
sns.barplot(
    data=avg_peaks_df,
    x='condition',
    y='peak_count',
    errorbar=('se', 2),
    color='skyblue',
    width=0.6
)
# Overlay individual exp_name averages as points
sns.stripplot(
    data=avg_peaks_df,
    x='condition',
    y='peak_count',
    color='black',
    size=7,
    jitter=True,
    alpha=0.7
)

# Add averages as text above each bar
ax = plt.gca()
grouped = avg_peaks_df.groupby('condition')['peak_count'].mean()
for i, (condition, avg) in enumerate(grouped.items()):
    ax.text(i, 0.1, f"avg: {avg:.2f}", ha='center', va='bottom', fontsize=10, rotation=90)

sns.despine()
plt.ylabel("Average number of peaks per cell")
plt.xlabel("")
plt.title("Average number of peaks per cell by condition (+-SEM)")
plt.tight_layout()

plt.savefig(r'D:\Peak finder\plots\avg_peaks_by_condition_barplot.pdf', bbox_inches='tight')

plt.show()


In [ ]:
df = pd.read_csv(r"D:\Peak finder\data\avg_peaks_per_exp.csv")

In [ ]:
#two way anova and tukey's hsd post hoc test

import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import matplotlib.pyplot as plt



# === Extract Two Factors from 'condition' ===
df[['gene', 'treatment']] = df['condition'].str.split(' - ', expand=True)

# === Two-Way ANOVA ===
model = ols('peak_count ~ C(gene) + C(treatment) + C(gene):C(treatment)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("\n=== Two-Way ANOVA Results ===")
display(anova_table)

# === Post Hoc: Tukey's HSD on Combined Groups ===
df['group'] = df['gene'] + ' - ' + df['treatment']

tukey = pairwise_tukeyhsd(endog=df['peak_count'],
                          groups=df['group'],
                          alpha=0.05)

print("\n=== Tukey HSD Post Hoc Test ===")
display(tukey.summary())